In [3]:
from pathlib import Path
import torch
import torch.nn as nn

gpu_id = 0
num_cores = 4
torch.cuda.set_device(gpu_id)
data_pth = Path('/home/alaa/Dropbox/BPHO Staff/USF/EM/training/trainsets/hr/')

In [4]:
from JigsawDataLoader import DataLoader
from JigsawNetwork import Network

In [5]:
ds = DataLoader(data_pth)

In [6]:
model = Network()